# Conway's Game of Life

In [5]:
import numpy as np
import pygame
import time
from genetic_algorithm import calculate_fitness_cell, select_mate, crossover_and_mutate

cellsX, cellsY = 200, 200
width, height = 800, 800
mutation_rate = 0.001

dimCW = width / cellsX
dimCH = height / cellsY

def next_generation(gameState):
    newGameState = np.copy(gameState)
    for x in range(cellsX):
        for y in range(cellsY):

            # Calculate fitness for the cell
            neighborhood = gameState[max(0, x-1):x+2, max(0, y-1):y+2]
            fitness_scores = np.zeros_like(neighborhood)
            
            for i in range(neighborhood.shape[0]):
                for j in range(neighborhood.shape[1]):
                    nx = (x-1+i) % cellsX
                    ny = (y-1+j) % cellsY
                    fitness_scores[i, j] = calculate_fitness_cell(gameState, nx, ny)
            
            # Select mate and apply crossover/mutate
            mate_x, mate_y = select_mate(neighborhood, fitness_scores)
            mate_state = gameState[(mate_x + x - 1) % cellsX, (mate_y + y - 1) % cellsY]
            newGameState[x, y] = crossover_and_mutate(gameState[x, y], mate_state, mutation_rate)
    return newGameState

pygame.init()
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Game of Life")

color_bg = 25, 25, 25
screen.fill(color_bg)

gameState = np.random.choice([0, 1], size=(cellsX, cellsY), p=[0.99, 0.01])

pause = False
running = True

generation = 0
while running:
    screen.fill(color_bg)
    time.sleep(0.1)

    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            pause = not pause

        if event.type == pygame.QUIT:
            running = False

        click = pygame.mouse.get_pressed()
        if sum(click) > 0:
            posX, posY = pygame.mouse.get_pos()
            celX, celY = int(np.floor(posX / dimCW)), int(np.floor(posY / dimCH))
            gameState[celX, celY] = not gameState[celX, celY]

    if not pause:
        gameState = next_generation(gameState)
        generation += 1

    for y in range(0, cellsY):
        for x in range(0, cellsX):
            center_x = (x) * dimCW
            center_y = (y) * dimCH
            radius = min(dimCW, dimCH) / 2 - 1

            if gameState[x, y] == 0:
                pygame.draw.circle(screen, (24, 24, 24), (int(center_x), int(center_y)), int(radius), width=1)
            else:
                pygame.draw.circle(screen, (255, 255, 255), (int(center_x), int(center_y)), int(radius), width=0)
            

    pygame.display.set_caption(f"Game of Life  -  Generation: {generation}")
    pygame.display.flip()

pygame.quit()
